In [1]:
import pandas as pd
from npi.npi import NPI
from npi.utils.utils import longprint

### Instantiate an NPI object

The NPI class provides convenient access to the NPI data housed in our shared folders.

The canonical way to obtain an instance of this class is via (where entities=1 is specified if you only want to work with individuals)

In [2]:
npi = NPI(entities=1)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_entity
* Metadata: {'func': 'get_entity', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_entity': '-code snipped-'}} 
* (identified) Called functions: ['get_entity']
* Cache found; loaded from ID 1587436437915440
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



### Display a list of data that can be currently managed by adding as an attibute

In [3]:
npi.display_gettable_attributes()

Gettable raw data:
 ['PLICNUM', 'PLICSTATE', 'entity', 'npideactdate', 'npireactdate', 'pcredential', 'pcredentialoth', 'pfname', 'pfnameoth', 'plname', 'plnameoth', 'ploccityname', 'plocline1', 'plocline2', 'plocstatename', 'ploctel', 'ploczip', 'pmname', 'pmnameoth', 'pnamesuffix', 'pnamesuffixoth', 'ptaxcode'] 


Gettable processed data:
 ['credentials', 'expanded_fullnames', 'fullnames', 'licenses', 'practitioner_type', 'removaldate', 'secondary_practice_locations']


### Retrieve all the data you need 

Hint: if you want to get rid of all the bulky output, run the below cell twice. The second time will be instantaneous and will clear the output.

In [12]:
npi.retrieve('removaldate')
npi.retrieve('fullnames')
npi.retrieve('practitioner_type')
npi.retrieve('plocstatename')
npi.retrieve('credentials')

### Construct a dataset of NPI removals that can be used to search for people

1. Get only doctors and advanced practice nurses for now

In [5]:
npi.practitioner_type.set_index('npi')[['MD/DO', 'CNM', 'CNS', 'CRNA', 'NP', 'Other APRN']]

,MD/DO,CNM,CNS,CRNA,NP,Other APRN
npi,,,,,,
1003000100,0,0,0,0,0,0
1003000126,1,0,0,0,0,0
1003000134,1,0,0,0,0,0
1003000142,1,0,0,0,0,0
1003000159,0,0,0,0,1,0
...,...,...,...,...,...,...
1992999833,0,0,0,0,0,0
1992999858,0,0,0,0,0,0
1992999866,0,0,0,0,0,0


In [6]:
s = npi.practitioner_type.set_index('npi')[['MD/DO', 'CNM', 'CNS', 'CRNA', 'NP', 'Other APRN']].sum(axis=1) > 0 
mds_aprns = s[s].reset_index().drop(columns=0)

2. Merge together the MD/APRN list with the deactivation list and their credentials

In [7]:
removed = npi.removaldate.merge(mds_aprns)
removed = removed.merge(npi.credentials)
removed = removed.drop(columns='pcredential_stripped')
removed

,npi,npideactdate,pcredential,cat
0,1003001660,2016-05-27,MD,MD/DO
1,1003003971,2019-11-04,MD,MD/DO
2,1003004326,2017-04-28,DO,MD/DO
3,1003006339,2014-04-30,MD,MD/DO
4,1003006479,2014-05-05,MD,MD/DO
...,...,...,...,...
50214,1992995732,2010-12-20,MD,MD/DO
50215,1992996169,2014-02-10,MD,MD/DO
50216,1992997316,2014-10-14,MD,MD/DO
50217,1992997761,2019-06-10,MD,MD/DO


3. Merge in the state locations in the month they are removed

Here we are selecting the last month observed for each NPI, for convenience

In [8]:
states = npi.plocstatename.groupby('npi', as_index=False).last().drop(columns='month')

In [9]:
removed = removed.merge(states)

4. Merge in full names

In [10]:
removed = removed.merge(npi.fullnames)

### 60 most recent removals

People can be duplicated if they have more than one value for fullname or credential. You can use this additional context to find people

In [11]:
removed.sort_values(['npideactdate', 'npi'])[-60:].head(60)

,npi,npideactdate,pcredential,cat,plocstatename,pfname,pmname,plname,pnamesuffix,othflag
13297,1245387414,2020-04-24,,NaN,MD,JULIA,L,SURRATT,,0
29588,1548508435,2020-04-24,MD,MD/DO,PA,PARVIZ,,KAMBIN,,0
37111,1689984676,2020-04-24,FNP,NP,MD,RITA,M,LEE,,1
37112,1689984676,2020-04-24,FNP,NP,MD,RITA,MARIE,ROBERTAZZI,,0
42141,1780662478,2020-04-24,DO,MD/DO,OH,PAUL,R,WEST,,0
49123,1912086810,2020-04-24,MD,MD/DO,FL,MARK,I,TAFEEN,,0
50329,1932212362,2020-04-24,MD,MD/DO,PA,RUDOLPH,EM,JANOSKO,,0
52180,1962846956,2020-04-24,MD,MD/DO,NY,ANTHONY,EDWARD,LANG,,0
18740,1346545365,2020-04-27,MD,MD/DO,PA,JOHN,WILLIAM,REINHART,,0
21000,1386932986,2020-04-27,MD,MD/DO,OK,GEORGE,FRED,MCDONNOLD,,0
